In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

from sklearn import metrics
from sklearn.preprocessing import label_binarize

import numpy as np
import pandas as pd

2024-01-30 19:22:32.866019: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 19:22:32.888328: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 19:22:32.888354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 19:22:32.889049: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 19:22:32.892860: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 19:22:32.893194: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
x_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_train.csv', low_memory=False)

In [3]:
x_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_test.csv', low_memory=False)

In [8]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Compute metrics
    precision = metrics.precision_score(y_test, y_pred_classes, average='weighted')
    accuracy = metrics.accuracy_score(y_test, y_pred_classes)
    f1 = metrics.f1_score(y_test, y_pred_classes, average='macro')
    auc = metrics.roc_auc_score(y_test, y_pred_classes, multi_class='ovr')

    print(f"Precision: {precision}\nAccuracy: {accuracy}\nF1 Score: {f1}\nAUC: {auc}")
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()

    fnr = fn / (fn + tp)
    tnr = tn / (tn + fp)
    fpr = fp / (fp + tn)
    tpr = tp / (tp + fn)


    # Printing the mean metrics
    print(f"FNR: {fnr}\nTNR: {tnr}\nFPR: {fpr}\nTPR: {tpr}")

In [6]:
input_shape = x_train.shape[1:]  # Determined by the shape of your training data
output_shape = len(np.unique(y_train))  # Assuming classification task

# Function to create the DNN model
def create_model(units=32):
    model = Sequential()
    model.add(Dense(units, input_shape=input_shape, activation='relu'))
    model.add(Dense(units, activation='relu'))
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrapping the model with KerasClassifier
model = KerasClassifier(model=create_model, verbose = 1, units = [32, 64, 128], batch_size = [1000, 1200, 1400], epochs = [15, 20, 25])

# Define the parameter grid for hyperparameter tuning
param_dist = {
    'units': [32, 64, 128],
    'batch_size': [1000, 1200, 1400],
    'epochs': [15, 20, 25]
}

# Setting up RandomizedSearchCV
# Setting up RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, 
                                   n_iter=10, cv=3, verbose=2)

# Perform hyperparameter tuning using x_val and y_val
random_search_result = random_search.fit(x_train, y_train)

# Best parameters
best_params = random_search_result.best_params_
print(f"Best Parameters: {best_params}")

# Train the optimal model with best parameters
optimal_model = create_model(units=best_params['units'])
optimal_model.fit(x_train, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/20


2024-01-30 19:24:20.358035: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


46/46 [==============================] - 0s 2ms/step - loss: 2.8010 - accuracy: 0.7147
Epoch 2/20
46/46 [==============================] - 0s 1ms/step - loss: 0.3415 - accuracy: 0.8206
Epoch 3/20
46/46 [==============================] - 0s 1ms/step - loss: 0.2824 - accuracy: 0.8569
Epoch 4/20
46/46 [==============================] - 0s 1ms/step - loss: 0.2585 - accuracy: 0.8722
Epoch 5/20
46/46 [==============================] - 0s 1ms/step - loss: 0.2378 - accuracy: 0.8812
Epoch 6/20
46/46 [==============================] - 0s 1ms/step - loss: 0.2115 - accuracy: 0.9039
Epoch 7/20
46/46 [==============================] - 0s 1ms/step - loss: 0.2105 - accuracy: 0.8937
Epoch 8/20
46/46 [==============================] - 0s 1ms/step - loss: 0.2076 - accuracy: 0.8930
Epoch 9/20
46/46 [==============================] - 0s 1ms/step - loss: 0.1949 - accuracy: 0.9025
Epoch 10/20
46/46 [==============================] - 0s 1ms/step - loss: 0.1736 - accuracy: 0.9230
Epoch 11/20
46/46 [===========

In [7]:
# Make predictions and evaluate the model
calculate_performance_metrics(x_test, y_test, optimal_model)

5480/5480 [==============================] - 2s 419us/step
[1 1 1 ... 1 1 1]
Precision: 0.91777337724721
Accuracy: 0.9163002378223005
F1 Score: 0.904919444990984
AUC: 0.9100836903285542
FNR: 0.07270761934289138
TNR: 0.892875
FPR: 0.107125
PR: 0.9272923806571086
